In [9]:
# Imports
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal
import os

print("✅ All imports successful")

✅ All imports successful


In [10]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")

print("✅ API key loaded")

✅ API key loaded


In [11]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


In [12]:
# IMPORTANT: Replace this path with your PDF file
file_path = r"C:\Users\DELL\Desktop\Customer_Support\Customer_support_chatbot\rag_files"

# Check if file exists
if not os.path.exists(file_path):
    print(f"⚠️ File not found: {file_path}")
    print("Please update the file_path variable with your PDF file.")
    print("\nFor this demo, we'll create sample documents instead...")
    
    # Create sample documents for demo
    from langchain_core.documents import Document
    pages = [
        Document(page_content="Nigeria Tax Bill, 2024", 
                metadata={"page": 1}),
        Document(page_content="Proteins are made of amino acids and perform many functions in cells.",
                metadata={"page": 2}),
        Document(page_content="DNA stores genetic information using four nucleotide bases.",
                metadata={"page": 3}),
    ]
    print("✅ Using sample documents for demo")
else:
    # Load the PDF
    # loader = PyPDFLoader(file_path)
    loader = DirectoryLoader(
    file_path,
    glob="**/*.pdf",
    loader_cls=PyPDFLoader
)
    pages = []
    
    # Load pages (async loading)
    async for page in loader.alazy_load():
        pages.append(page)
    
    print(f"✅ Loaded {len(pages)} pages from PDF")

✅ Loaded 1472 pages from PDF


In [13]:
# Create text splitter (Module 2 knowledge!)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Characters per chunk
    chunk_overlap=100     # Overlap to preserve context
)

# Split documents
doc_splits = text_splitter.split_documents(pages)

print(f"✅ Created {len(doc_splits)} chunks")
print(f"\nSample chunk:")
print(f"{doc_splits[0].page_content[:200]}...")

✅ Created 3015 chunks

Sample chunk:
A Practical Introduction to
Python Programming
Brian Heinold
Department of Mathematics and Computer Science
Mount St. Mary’s University...


In [14]:
embeddings = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    api_key = openai_api_key
)

print(" Embedding model initialized ♻")

 Embedding model initialized ♻


In [15]:
# Create Chroma vector store
chroma_path = "./chroma_db_agentic_rag"

# Create vector store from documents
vectorstore = Chroma(
    collection_name="agentic_rag_docs",
    persist_directory=chroma_path,
    embedding_function=embeddings
)

# Add documents
vectorstore.add_documents(documents=doc_splits)

print(f"✅ Vector store created with {len(doc_splits)} chunks")
print(f"   Persisted to: {chroma_path}")

✅ Vector store created with 3015 chunks
   Persisted to: ./chroma_db_agentic_rag


In [16]:

vectorstore = Chroma(
     collection_name="agentic_rag_docs",
     persist_directory="./chroma_db_agentic_rag",
     embedding_function=embeddings
)

In [17]:
# Test the vector store
test_query = "What is python?"
test_results = vectorstore.similarity_search(test_query, k=2)

print(f"Query: {test_query}")
print(f"\nTop result:")
print(f"{test_results[0].page_content[:200]}...")
print(f"\n✅ Retrieval working!")

Query: What is python?

Top result:
1.8Why Python?
1. a.The language was named after Monty Python. Just for fun, the official documentation makes subtle
references to Monty Python comedy sketches.
2. b.Python's first release was in 1991...

✅ Retrieval working!


In [18]:
@tool
def retrieve_documents(query: str) -> str:
    """
    Search for relevant documents in the knowledge base.
    
    Use this tool when you need information from the document collection
    to answer the user's question. Do NOT use this for:
    - General knowledge questions
    - Greetings or small talk
    - Simple calculations
    
    Args:
        query: The search query describing what information is needed
        
    Returns:
        Relevant document excerpts that can help answer the question
    """
    # Use MMR (Maximum Marginal Relevance) for diverse results
    retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 5, "fetch_k": 10}
    )
    
    # Retrieve documents
    results = retriever.invoke(query)
    
    if not results:
        return "No relevant documents found."
    
    # Format results
    formatted = "\n---\n".join([f"Source: {r.metadata.get('source')}\n{r.page_content[:200]}..." 
                                for r in results]
    )
    
    
    return formatted

print("✅ Retrieval tool created")

✅ Retrieval tool created


In [19]:
system_prompt = SystemMessage(content="""You are CodeSensei, a Python programming assistant designed to help users learn and understand Python clearly and accurately.

You have access to a knowledge base containing the following textbooks:
- Python for Dummies
- Python for Everybody
- A Practical Introduction to Programming
- Introduction to Python Programming
- Pyhton Programming Notes

RETRIEVE FROM THE KNOWLEDGE BASE WHEN:
- The user asks about Python syntax, keywords, or language rules
- The user asks how to use Python functions, classes, or standard libraries
- The user asks “how to” or “what is” questions related to Python programming
- The user requests Python code examples or step-by-step explanations
- The question is likely covered in beginner or introductory Python textbooks

ANSWER DIRECTLY WITHOUT RETRIEVAL WHEN:
- The user greets you or engages in casual conversation
- The question is not related to Python programming
- The question involves simple general knowledge or basic arithmetic
- The answer does not require textbook-based explanation

WHEN YOU RETRIEVE:
- Use the document retrieval tool to find relevant passages
- Base your answer strictly on the retrieved content
- Cite the source book title and page or section when available
- Do not invent information not found in the documents

RESPONSE STYLE:
- Be clear, concise, and beginner-friendly
- Explain concepts step by step when needed
- Include short, correct Python code snippets where helpful

""")

print("✅ System prompt configured")

✅ System prompt configured


In [20]:
# Bind tool to LLM
tools = [retrieve_documents]
llm_with_tools = llm.bind_tools(tools)

def assistant(state: MessagesState) -> dict:
    """
    Assistant node - decides whether to retrieve or answer directly.
    """
    messages = [system_prompt] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """
    Decide whether to call tools or finish.
    """
    last_message = state["messages"][-1]
    
    if last_message.tool_calls:
        return "tools"
    return "__end__"

print("✅ Agent nodes defined")

✅ Agent nodes defined


In [21]:
# Build graph
builder = StateGraph(MessagesState)

# Add nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    should_continue,
    {"tools": "tools", "__end__": END}
)
builder.add_edge("tools", "assistant")

# Add memory
memory = MemorySaver()
agent = builder.compile(checkpointer=memory)

print("✅ Agentic RAG system compiled")

✅ Agentic RAG system compiled


In [22]:
def test_agent(user_input: str, thread_id: str = "default_session"):
    """
    Improved query function with clearer output.
    """
    print(f"\n{'='*70}")
    print(f"👤 User: {user_input}")
    print(f"{'='*70}\n")

    result = agent.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config={"configurable": {"thread_id": thread_id}}
    )

    # Check what happened
    used_retrieval = False
    final_answer = None

    for message in result["messages"]:
        if isinstance(message, AIMessage):
            if message.tool_calls:
                used_retrieval = True
                print(f"🔍 Agent: [Calling retrieval tool...]")
            if message.content and not message.tool_calls:
                final_answer = message.content

    # Always print final answer
    if final_answer:
        print(f"🤖 Agent: {final_answer}")
    else:
        print(f"⚠️ No response generated after retrieval!")

    # Summary
    print(f"\n📊 Decision: {'USED RETRIEVAL' if used_retrieval else 'ANSWERED DIRECTLY'}")
    print(f"{'='*70}\n")



In [24]:
tests = [
    ("What is a variable in Python?", True),
    ("How do I write a for loop in Python?", True),
    ("What is the difference between list and tuple?", True),
    ("How do I install Python on Windows?", True),
    ("What does the print() function do?", True),
]
results = []
for query, expected in tests:
    actual = test_agent(query, f"test_{len(results)}")
    correct = actual == expected
    results.append(correct)
    print(f"Expected: {'RETRIEVE' if expected else 'DIRECT'} | {':' if correct else ':x:'}")




👤 User: What is a variable in Python?

🔍 Agent: [Calling retrieval tool...]
🤖 Agent: In Python, a variable is a symbolic name that is used to refer to a value stored in memory. It acts as a container for data, allowing you to store, manipulate, and retrieve values throughout your program. Here are some key points about variables in Python:

1. **Dynamic Typing**: You do not need to declare the type of a variable when you create it. Python automatically determines the type based on the value assigned to it. For example:
   ```python
   my_variable = 42      # my_variable is an integer
   my_variable = "Hello" # now my_variable is a string
   ```

2. **Naming Rules**: Variable names can include letters, numbers, and underscores, but must start with a letter or underscore. They cannot contain spaces or special characters. For example:
   ```python
   valid_variable_name = 10
   _anotherVariable = 20
   ```

3. **Assignment**: You assign a value to a variable using the equals sign (`=`). 

In [26]:
def interactive_mode():
    print("\n" + "="*60)
    print(" Welcome To CodeSensei")
    print("Type 'exit' to quit")
    print("="*60)
    
    thread_id = "session 1"
    while True:
        query = input("\n: You: ").strip()
        if query.lower() in ["exit", "quit"]:
            print("Making Your Code Better, One Line at a Time")
            break
        
        result = agent.invoke(
            {"messages": [HumanMessage(content=query)]},
            config={"configurable": {"thread_id": thread_id}}
        )
        print(f" {result['messages'][-1].content}")

interactive_mode()


 Welcome To CodeSensei
Type 'exit' to quit
 To write functional code in Python, you can follow several key concepts associated with functional programming. Here are some important aspects:

### 1. First-Class Functions
In Python, functions are first-class citizens. This means you can treat functions like any other variable: you can assign them to variables, pass them as arguments to other functions, and return them from other functions.

**Example:**
```python
def greet(name):
    return f"Hello, {name}!"

greeting_function = greet
print(greeting_function("Alice"))  # Output: Hello, Alice!
```

### 2. Higher-Order Functions
Higher-order functions are functions that can take other functions as arguments or return them as results. This allows for greater flexibility and reusability in your code.

**Example:**
```python
def apply_function(func, value):
    return func(value)

result = apply_function(lambda x: x * 2, 5)  # Passing a lambda function
print(result)  # Output: 10
```

### 3. 